In [1]:
import numpy
np.random?

Object `np.random` not found.


In [34]:
from setup import *
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import random

def choose(n,k):
    try: 
        return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
    except:
        return 0


N=4
num_edges = choose(N,2)
print(num_edges)
edge_idx_to_pair = list(it.combinations(range(N),2))
edge_idx_to_pair = np.array(edge_idx_to_pair).astype('int16')
#print(edge_idx_to_pair)
edge_pair_to_idx = (num_edges/2*np.eye(N)).astype('uint16')
for (idx,e) in enumerate(edge_idx_to_pair):
    print(e,type(e))
    edge_pair_to_idx[tuple(e)] = idx
edge_pair_to_idx += edge_pair_to_idx.T


idx = 4
e = edge_idx_to_pair[idx]
print(idx, edge_pair_to_idx[tuple(e)])

e = [1,3]
idx = edge_pair_to_idx[tuple(e)]
print(e, edge_idx_to_pair[idx])


edge_pair_to_idx_gpu = gpuarray.to_gpu(edge_pair_to_idx.ravel())
print(edge_pair_to_idx_gpu)
edge_idx_to_pair_gpu = gpuarray.to_gpu(edge_idx_to_pair.ravel())
print(edge_idx_to_pair_gpu)
idx = 3
edge_idx_to_pair_gpu[2*idx:2*(idx+1)]
pair = [1,3]
edge_pair_to_idx_gpu[pair[0]*N+pair[1]]




6
[0 1] <class 'numpy.ndarray'>
[0 2] <class 'numpy.ndarray'>
[0 3] <class 'numpy.ndarray'>
[1 2] <class 'numpy.ndarray'>
[1 3] <class 'numpy.ndarray'>
[2 3] <class 'numpy.ndarray'>
4 4
[1, 3] [1 3]
[6 0 1 2 0 6 3 4 1 3 6 5 2 4 5 6]
[0 1 0 2 0 3 1 2 1 3 2 3]


array(4, dtype=uint16)

In [2]:
from setup import *

Ramsey = [3,4]
num_vertices = 8
num_threads = 5
beta = 1
num_steps = 5*10**2
#main(Ramsey, num_vertices, num_threads, beta, num_steps)

In [ ]:
gpu_Ramsey = """
    __global__ void Ramsey(int *Big_Vector, int length_Big_Vector,int length_sub_graph,int *sub_graph, int *problems)
    {
        int id = threadIdx.x + blockIdx.x*blockDim.x;
        if(id==0)
        {
            if(Big_Vector[subgraph[0]] == 1)
            {
                for(int i = 1; i < length_sub_graph; i++)
                {
                    if(Big_Vector[subgraph[i]] == 0)
                    {
                        break;
                    }
                    else if(i == length_sub_graph - 1 && Big_Vector[subgraph[i]] == 1)
                    {
                        atomicAdd(&problems[0], 1);
                    }
                }
            }
            else if(Big_Vector[subgraph[0]] == 0)
            {
                for(int i = 1; i < length_sub_graph; i++)
                {
                    if(Big_Vector[subgraph[i]] == 1)
                    {
                        break;
                    }
                    else if(i == length_sub_graph - 1 && Big_Vector[subgraph[i]] == 0)
                    {
                        atomicAdd(&problems[0], 1);
                    }
                }
            }
        }
    }
    """


In [6]:
def choose(n,k):
    try: 
        return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
    except:
        return 0
num_vertices = 5
#l := length of the subgraph we're trying to create
l = 2
#This is how we're currently generating the cliques and what we're going to compare our algorithm to
cliques = it.combinations(range(num_vertices),l)
#This is a 2-dimensional array that holds all of the values for us to search through
choose_table = np.array(pd.DataFrame([[choose(i,j) for i in range(num_vertices+1)] for j in range(1,1+l)]).T)
#list of all of the arrays generated by the new algorithm
result = []
#list of all of the arrrays generated by it.combinations
original = []
for N in range(choose(num_vertices,l)):
    index = np.ones(l)*-1
    for i in range(1,l+1):
        k = l - i +1
        j = int(0) 
        while choose_table[j+1][k-1] <= N:
            j+=1
        index[k-1] = int(j)
        N = N - choose(j,k)
    result.append(np.array(index))
    original.append(np.array(next(cliques)))
#number of elements of result also in original
#good = 0
#for i in range(len(result)):    
    #original[k]==result[i] returns a list of True's and False's which counted as 1's and 0's when the list is summed
    #up where each True represents the corresponding elements of original and result being identical So, if the sum 
    #of the list equals l, then the kth element of original is identical to the ith element of result and the max value
    #of this list comprehension is 1. Else, the max value will be 0. Note, recall it.combinations won't output two identical
    #arrays so, for each element of result, either it is identical to one element in original or none of them.
#    if(np.max([int(np.sum(original[k]==result[i])==l) for k in range(len(original))]))==1:
#        good += 1
    #else:
    #    print('bad')
bill = list(zip(original, result))
print(bill)

6
[(array([0, 1]), array([ 0.,  1.])), (array([0, 2]), array([ 0.,  2.])), (array([0, 3]), array([ 1.,  2.])), (array([0, 4]), array([ 0.,  3.])), (array([1, 2]), array([ 1.,  3.])), (array([1, 3]), array([ 2.,  3.])), (array([1, 4]), array([ 0.,  4.])), (array([2, 3]), array([ 1.,  4.])), (array([2, 4]), array([ 2.,  4.])), (array([3, 4]), array([ 3.,  4.]))]


In [ ]:
if np.array_equal(np.array(range(3)), np.array(range(4))) == False:
    print("Cool Beans")

In [ ]:
def main(Ramsey, num_vertices, num_threads, beta, num_steps):

    def choose(n,k):
        return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))

    Ramsey.sort()
    Ramsey = Ramsey[::-1]
    num_colors = len(Ramsey)
    Colors = np.arange(num_colors)
    Vertices = np.arange(num_vertices)
    Edges = list(it.combinations(Vertices,2))
    #reverse lookup for edges below.  Eg if slot 3 above contains edge (2,5), the the dict below has entry (2,5):3
    Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges)) 
    num_edges = len(Edges)

    
    #This code evenly distributes work to threads so that they all finish at about the same time
    vertices_per_clique = Ramsey
    edges_per_clique = [choose(v,2) for v in vertices_per_clique] 
    cliques_per_color = np.asarray([choose(num_vertices,v) for v in vertices_per_clique])
    edges_per_color = edges_per_clique * cliques_per_color
    computations_per_color = edges_per_color / edges_per_color.sum()

    threads_per_color = np.ones(num_colors).astype('int')
    remaining_threads = num_threads - threads_per_color.sum()
    threads_per_color += np.floor(remaining_threads * computations_per_color).astype('int')
    
    cliques_per_thread = np.ceil(cliques_per_color / threads_per_color).astype('int')
    
    #This object below tells each thread which color and set of cliques to monitor.
    #It is a list with an entry for each color.  Each entry is itself a 3D-matrix.
    #In the matrix for color c, the (i,j,k) entry is the edge_idx of 
    #the kth edge in the jth clique monitored by the ith thread associated to color c.
    #Note: The number of threads for a color might not evenly divide the number of cliques for that color.
    #If so, some threads will have 1 less clique than others.  We fill this last entry with -1.
    #Later, such a clique will be monochromatic with color = num_colors + 1.
    #This color does not appear elsewhere.  Hence we easily spot such a situation and these cliques
    #will not be counted as problems.
    assign_Threads_to_Cliques = [-1*np.ones([threads_per_color[color],
                                             cliques_per_thread[color],
                                             edges_per_clique[color]],
                                            dtype='int')
                                for color in Colors]

    for color, clique_size in enumerate(Ramsey):        
        num_cliques = cliques_per_color[color]
        num_threads = threads_per_color[color]
        #Creates a generator to produce all cliques (the list of vertices).
        Cliques = it.combinations(Vertices,clique_size)

        #Makes the vector [0,1,2,...,num_threads-1,0,1,2,...,num_threads-1,....] of length num_cliques        
        assign_Cliques_to_Threads = np.arange(num_cliques) % num_threads
        #randomizes assignment, but maintains clique counts
        np.random.shuffle(assign_Cliques_to_Threads)    

        #A vector that knows where the next open slot on each thread is
        next_open_slot = np.zeros(num_threads,dtype='int')
        for clique_Vertices, thread in zip(Cliques,assign_Cliques_to_Threads):
            #Gets the list of edges in this clique
            clique_Edges = list(it.combinations(clique_Vertices,2))
            #Converts it to edge_idx
            clique_Edges_idx = [Edges_idx[edge] for edge in clique_Edges]        
            #Writes it to the correct thread and next open slot on that thread
            assign_Threads_to_Cliques[color][thread,next_open_slot[thread]] = clique_Edges_idx
            next_open_slot[thread] += 1


    #print("edges per clique");print(edges_per_clique);print("cliques per color");print(cliques_per_color);print("edges per color");print(edges_per_color);print("threads per color");print(threads_per_color)
    """
    for color in Colors:        
        print("color");print(color);print("threads per color");print(threads_per_color[color]);print("cliques per thread");print(cliques_per_thread[color]);print("edges per clique");print(edges_per_clique[color])    
        #for thread in range(threads_per_color[color]):
        #    display(assign_Threads_to_Cliques[color][thread])
        display(np.array(assign_Threads_to_Cliques[color]))#[thread]))
    """

    def find_problems(Problems,num_problems_per_color,printout=False):    
        for color in Colors:        
            X = coloring[assign_Threads_to_Cliques[color]]        
            Problems[color] = np.all(X == color,axis=2)        
            num_problems_per_color[color] = Problems[color].sum()
            if printout == True:
                print(color)
                XF = pd.DataFrame(X.reshape(-1, X.shape[-1]))
                ProblemsF = Problems[color].reshape(-1)
                XF['problem'] = ProblemsF
                display(XF)
                display(XF[ProblemsF])
        return num_problems_per_color.sum()

    if num_colors == 2:
        print("2 colors")
        def flip():        
            coloring[edge_idx] += 1
            coloring[edge_idx] %= 2        
            return
    else:
        print("more than 2 colors")
        nc = num_colors-1
        def flip():
            new_color = np.random.randint(0,nc)
            if new_color >= coloring[edge_idx]:
                new_color += 1
            coloring[edge_idx] = new_color
            return

    coloring = np.random.choice(Colors, size=num_edges+1, replace=True)
    coloring[-1] = 1 + Colors[-1]
    Problems_current =  [np.zeros(assign_Threads_to_Cliques[color].shape[:-1]).astype('bool') for color in Colors]
    num_problems_per_color_current = np.zeros(num_colors).astype('int')
    Problems_proposed = [np.zeros(assign_Threads_to_Cliques[color].shape[:-1]).astype('bool') for color in Colors]
    num_problems_per_color_proposed = np.zeros(num_colors).astype('int')
    double = []
    num_problems_current = find_problems(Problems_current, num_problems_per_color_current)#, printout=True)
    step = -1
    print(step,num_problems_per_color_current,num_problems_current)
    for step in range(num_steps):
        print()        
        edge_idx = np.random.randint(num_edges)
        edge_color_current = coloring[edge_idx]    
        flip()
        num_problems_proposed = find_problems(Problems_proposed, num_problems_per_color_proposed)#, printout=True)
        double_check = find_problems(Problems_proposed, num_problems_per_color_proposed)
        double.append(np.sum(num_problems_proposed-double_check))
        num_problems_diff = num_problems_current - num_problems_proposed
        print(num_problems_diff)
        if num_problems_diff >= 0:
            #accept the move
            #print("Proposed is better.  Accepting.")
            num_problems_per_color_current[:] = num_problems_per_color_proposed[:]
            num_problems_current = num_problems_proposed
        else:            
            accept = np.exp(beta * num_problems_diff)            
            r = np.random.random()
            #print("Proposed is worse")
            #print("But I will accept it anyway I draw a number less than %.3f.  I drew %.3f." % (accept,r))            
            if r <= accept:
                #accept the move
            #    print("So I accept the move even though it is worse.")
                num_problems_per_color_current[:] = num_problems_per_color_proposed[:]
                num_problems_current = num_problems_proposed
            else:
                #reject the move
            #    print("So I reject.")
                coloring[edge_idx] = edge_color_current
        print(step,coloring[:-1],num_problems_per_color_current,num_problems_current)
        if num_problems_current == 0:
            break
    num_problems_current = find_problems(Problems_current, num_problems_per_color_current, printout=True)
    print("Double", np.mean(double), np.var(double))
    #print(step,num_problems_per_color_current,num_problems_current)
    return coloring

In [ ]:
import networkx as nx

def choose(n,k):
    return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))

#Ramsey.sort()
#Ramsey = Ramsey[::-1]
#num_colors = len(Ramsey)
#Colors = np.arange(num_colors)
num_vertices = 10
Vertices = np.arange(num_vertices)
Edges = list(it.combinations(Vertices,2))
Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges))
num_edges = len(Edges)

G=nx.Graph()
G.add_nodes_from(list(range(num_vertices)))
G.nodes()
G.add_edges_from(Edges)
nx.draw(G)
plt.show()

In [ ]:
edge_idx = np.random.randint(num_edges)
def flip1():
    coloring[edge_idx] = (coloring[edge_idx] + np.random.randint(1,num_colors)) % num_colors
    #return(coloring[edge_idx])

def flip2():
    new_color = np.random.randint(num_colors-1)
    if new_color >= coloring[edge_idx]:
        new_color += 1
    coloring[edge_idx] = new_color
    #return(coloring[edge_idx])

#for step in range(num_steps):    
#    print(coloring[edge_idx]==flip1())
%timeit(flip1())
%timeit(flip2())
    
    
    

In [ ]:
    e = np.random.randint

In [ ]:
    e = np.random.choice

In [ ]:
        A = pd.merge

In [ ]:
np.all?

In [ ]:
from setup import *

def choose(n,k):
    return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))

Ramsey = [3,4]
num_vertices = 8
num_threads = 5

Ramsey = np.sort(Ramsey)
num_colors = len(Ramsey)
Colors = np.arange(num_colors)
Vertices = np.arange(num_vertices)
Edges = list(it.combinations(Vertices,2))
Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges))
num_edges = len(Edges)

vertices_per_clique = Ramsey
edges_per_clique = [choose(v,2) for v in vertices_per_clique] 
cliques_per_color = np.asarray([choose(num_vertices,v) for v in vertices_per_clique])
edges_per_color = edges_per_clique * cliques_per_color
threads_per_color = np.floor(num_threads * (edges_per_color / edges_per_color.sum())).astype('int')
cliques_per_thread = np.ceil(cliques_per_color / threads_per_color).astype('int')
assign_Threads_to_Cliques = [[[[-1
                                for _ in range(edges_per_clique[color])]
                                for _ in range(cliques_per_thread[color])]
                                for _ in range(threads_per_color[color])]
                                for color in Colors]

for color, clique_size in enumerate(Ramsey):    
    num_cliques = cliques_per_color[color]
    num_threads = threads_per_color[color]
    Cliques = it.combinations(Vertices,clique_size)
    
    assign_Cliques_to_Threads = np.arange(num_cliques) % num_threads 
    np.random.shuffle(assign_Cliques_to_Threads)    

    next_open_slot = np.zeros(num_threads,dtype='int')
    for clique_Vertices, thread in zip(Cliques,assign_Cliques_to_Threads):
        clique_Edges = list(it.combinations(clique_Vertices,2))
        
        clique_Edges_idx = [Edges_idx[edge] for edge in clique_Edges]        
        
        assign_Threads_to_Cliques[color][thread][next_open_slot[thread]] = clique_Edges_idx
        
        next_open_slot[thread] += 1

"""
print("edges per clique");print(edges_per_clique);print("cliques per color");print(cliques_per_color);print("edges per color");print(edges_per_color);print("threads per color");print(threads_per_color)
for color in Colors:        
    print("color");print(color);print("threads per color");print(threads_per_color[color]);print("cliques per thread");print(cliques_per_thread[color]);print("edges per clique");print(edges_per_clique[color])    
    #for thread in range(threads_per_color[color]):
    #    display(assign_Threads_to_Cliques[color][thread])
    display(np.array(assign_Threads_to_Cliques[color]))#[thread]))
"""

def count_issues(coloring):
    for color in Colors:
        for thread in range(threads_per_color[color]):            
            #for clique in Cliques:
            #    coloring[clique]
            X = np.array(assign_Threads_to_Cliques[color][thread])
            print(type(X))
            print(coloring[X])

            #print(type(assign_Threads_to_Cliques[color][thread]))
            #print(coloring[assign_Threads_to_Cliques[color][thread]])
            

coloring = np.random.choice(Colors, size=num_edges+1, replace=True)
coloring[-1] = 1 + Colors[-1]
print(coloring)
count_issues(coloring)

In [ ]:
assign_threads_to_cliques = []
#assign_threads_to_cliques = [np.full([threads_per_color[c],cliques_per_thread[c],edges_per_clique[c]],np.nan)
#                            for c in Colors]
           

#assign_threads_to_cliques = [[[np.full(edges_per_clique[color], np.nan)
#                               for _ in range(cliques_per_thread[color])]
#                               for _ in range(threads_per_color[color])]
#                               for color in Colors]
#assign_threads_to_cliques = [np.full([threads_per_color[c],cliques_per_thread[c],edges_per_clique[c]], np.nan, dtype='int')
#                               for c in Colors]
#assign_threads_to_cliques = [np.empty([threads_per_color[c],cliques_per_thread[c],edges_per_clique[c]], dtype='int')
#                               for c in Colors]

#print(assign_threads_to_cliques)
for color, clique_size in enumerate(Ramsey):
    num_cliques = cliques_per_color[color]
    num_threads = cliques_per_thread[color]
    Cliques = it.combinations(Vertices,clique_size)
    
    
    assign_cliques_to_threads = np.arange(num_cliques) % num_threads
    np.random.shuffle(assign_cliques_to_threads)
    #assign_threads_to_cliques.append([])

    next_open_slot = np.zeros(threads_per_color[color],dtype='int')
    for clique_Vertices, thread in zip(Cliques,assign_cliques_to_threads):
        clique_Edges = np.array(list(it.combinations(clique_Vertices,2)))
        display(clique_Edges)
        print(Edges in clique_Edges)

In [ ]:
        
        #assign_threads_to_cliques[color][thread,clique] = clique#.append(clique)
        #display(assign_threads_to_cliques[color][thread])
        #assign_threads_to_cliques[color][thread][next_open_slot[thread]] = Edges
        #next_open_slot[thread] +=1
        #display(assign_threads_to_cliques[color][thread])
        #display(next_open_slot)
        
    for thread in assign_threads_to_cliques[color]:
        print(thread)
        print()

In [ ]:
x = np.asarray(assign_threads_to_cliques[0])
x.shape

In [ ]:

    
print(coloring)

In [ ]:
a=5
for i in range(3):
    b is a
    b += b+1
print(a)
b is a
b += 1
print(a is b)

In [ ]:
l = []
display(l)
l.append([1,2,3])
display(l)
l.append([1,2,3])
display(l)